## Quantify hexapod motion fails
### This runs multiple images

Craig Lage  25-Jul-25

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pickle as pkl
from lsst.daf.butler import Butler
from lsst.summit.utils.efdUtils import makeEfdClient
from lsst.summit.utils.simonyi.mountAnalysis import calculateMountErrors, plotMountErrors
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.summit.utils.butlerUtils import getExpRecordFromDataId

In [ ]:
client = makeEfdClient()
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation',
                                              'LSSTCam/runs/nightlyValidation/20250425/w_2025_17/DM-50157'])

In [ ]:
startDay = 20250718
endDay = 20250718

rmss = {}
dayObs = startDay
while dayObs <= endDay:
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", 
                where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"):
        exposureList.append([record.id, record])
    exposureList.sort(key=lambda x: x[0])
    print(len(exposureList))
    for [id,record] in exposureList:
        if record.observation_type not in ['science', 'acq']:
            continue
        try:
            (mountErrors, mountData) = calculateMountErrors(record, client)
            rmss[record.id] =[mountErrors.camHexRms, mountErrors.m2HexRms]
            print(f"{record.id} succeeded!")
        except:
            print(f"{record.id} failed!")
            continue
    dayObs = calcNextDay(dayObs)

filename = "/home/c/cslage/u/Hexapods/data/failures_current_20250718.pkl"
with open(filename, 'wb') as f:
    pkl.dump(rmss, f)



In [ ]:
filename = "/home/c/cslage/u/Hexapods/data/failures_current_20250718.pkl"
with open(filename, 'rb') as f:
    rmss_current = pkl.load(f)
print(len(rmss_current))
      
filename = "/home/c/cslage/u/Hexapods/data/failures_20250718.pkl"
with open(filename, 'rb') as f:
    rmss_new = pkl.load(f)
print(len(rmss_new))

In [ ]:
cam_news = []
cam_olds = []
m2_news = []
m2_olds = []
for key in rmss_current.keys():
    if key not in rmss_new.keys():
        continue
    [cam_old, m2_old] = rmss_current[key]
    [cam_new, m2_new] = rmss_new[key]
    cam_news.append(cam_new)
    cam_olds.append(cam_old)  
    m2_news.append(m2_new)
    m2_olds.append(m2_old)        

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(10,5))
plt.suptitle(f"DM-52072 - New vs old Hexapod Errors")
plt.subplots_adjust(wspace=0.3, hspace=0.6)
axs[0][0].set_title("CamHex RMS")
axs[0][0].scatter(cam_olds, cam_news)
axs[0][1].set_title("M2Hex RMS")
axs[0][1].scatter(m2_olds, m2_news)
axs[1][0].set_title("CamHex RMS")
axs[1][0].scatter(cam_olds, cam_news)
axs[1][0].set_xlim(0, 0.1)
axs[1][0].set_ylim(0, 0.1)
axs[1][1].set_title("M2Hex RMS")
axs[1][1].scatter(m2_olds, m2_news)
axs[1][1].set_xlim(0, 0.1)
axs[1][1].set_ylim(0, 0.1)
for i in range(2):
    for j in range(2):
        axs[i][j].set_xlabel("Current")
        axs[i][j].set_ylabel("New - DM-52072")

saveFilename = f"/home/c/cslage/u/MTMount/mount_plots/DM-52072_New_vs_Old_20250718.png"
plt.savefig(saveFilename)


In [ ]:
pdf = PdfPages(f"/home/c/cslage/u/MTMount/mount_plots/Hex_RMS_Failures_{startDay}_{endDay}.pdf")
fig = plt.figure(figsize=(12, 10))
count = 0
for key in rmss.keys():
    [cam_rms, m2_rms] = rmss[key]
    if (cam_rms > 2.0) or (m2_rms > 2.0):
        count += 1
        print(key, cam_rms, m2_rms)
        dataId = {'exposure':key, 'instrument':'LSSTCam'}
        expRecord = getExpRecordFromDataId(butler, dataId)
        (mountErrors, mountData) = calculateMountErrors(expRecord, client)
        plotMountErrors(mountData, mountErrors, figure=fig) 
        fig.suptitle(f"expId = {key}", y=0.98)
        pdf.savefig(fig)
        fig.clf()
pdf.close()

print(f"There were {count} fails")        